# Data Cleaning

Notes: 
 - Raw data is in `../raw_data/soc99.csv`
 - Clean the data and store it in `./clean_data`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Read data
df99 = pd.read_csv(r"../raw_data/soc99.csv")
filename = r'.\soc{yrnum}.xls'
yearnum = np.array(['00', '01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18'])
data = df99
for i in range(0,19):
    df = pd.read_excel(filename.format(yrnum = yearnum[i]))
    df,data = df.align(data, join = 'inner', axis =1)
    data = pd.concat([data,df], axis = 0)

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [4]:
# Table with variable names
variables = pd.read_excel(r"./1.2.Variables.xlsx")

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [ ]:
dt = data

# Replace Columns name abbreviations with full names

In [ ]:
# Creating a dictionary with the variable names and their descriptions
variable_descriptions = {
    name: descr 
    for name, descr 
    in zip(
        list(variables['Variable']),
        list(variables['Description'])
    )
}

In [ ]:
# Replacing the variables name (Abbreviations) with their descriptions (Full Names)
dt.columns = [variable_descriptions[c] for c in dt.columns]

In [ ]:
dt.describe().T

# Missing Data

Note: **Some** of the values have `zero` for `not reported`.

In [ ]:
# Remove variables that can have "o = Not reported"
variables_with_not_reported = list(
    variables[
        variables['Possible Values'].str.contains('0 = Not reported')
    ]['Description']
)
for c in dt.columns:
    if c in variables_with_not_reported:
        print(f"Cleaning zero values in {c}")
        dt[c] = dt[c].replace(to_replace=0, value=np.nan)

In [ ]:
# Remove variables that can have "9 = Not reported"
variables_with_not_reported = list(
    variables[
        variables['Possible Values'].str.contains('9 = Not reported')
    ]['Description']
)
for c in dt.columns:
    if c in variables_with_not_reported:
        print(f"Cleaning zero values in {c}")
        dt[c] = dt[c].replace(to_replace=0, value=np.nan)

In [ ]:
# for the area vars and price vars, replace with nan if value = 0
area_vars = ['SquareFootAreaoftheLot', 'SquareFootAreaofFinishedBasement', 'SquareFootAreaofHouse']
price_vars = ['SalesPrice', 'ContractPrice']
for c in area_vars + price_vars:
    dt[c] = dt[c].replace(to_replace=0, value=np.nan)

In [ ]:
count_missing_values = pd.DataFrame((len(dt) - dt.count()).sort_values(ascending=False))
count_missing_values.columns = ['Number of Missing Values']
count_missing_values['% Missing Values'] = count_missing_values['Number of Missing Values'] / len(dt) * 100
count_missing_values[count_missing_values['Number of Missing Values'] > 0]

In [ ]:
dt.to_csv(r"./clean_dataComplete.csv")

In [ ]:
# I am still testing this ~ Gasser
# Trying to drop variables one after the other based on how much of the data remaining will be not NA
tmp_dt = dt.copy()
drop_x = []
drop_y = []
for i in range(len(dt.columns) - 1):
    initial_length = len(tmp_dt)
    sel = {c: len(tmp_dt.drop(columns=[c,]).dropna()) for c in tmp_dt.columns}
    choice = sorted(sel, key=sel.get, reverse=True)[0]
    tmp_dt = tmp_dt.drop(columns=[choice,])
    print(f"Dropping {choice}. Saving {sel[choice]} entries! Size after dropping na = {len(tmp_dt.dropna())}")
    drop_x.append(choice)
    drop_y.append(len(tmp_dt.dropna()))
drop_x.reverse()
drop_y.reverse()
y_pos = np.arange(len(drop_x))
plt.figure(figsize=(5, 15))
plt.barh(
    y_pos, 
    drop_y,
    align='center', 
    alpha=0.5
)
plt.yticks(y_pos, drop_x)
plt.xlabel("Entries remaining after dropna()")
plt.ylabel("Variable to drop in sequence")
plt.show()

In [ ]:
categorical = [c for c in dt.columns if len(dt[c].unique()) < 20]
categorical

In [ ]:
Numeric = [c for c in dt.columns if len(dt[c].unique()) > 20]
Numeric

In [ ]:
dt.head()